In [ ]:
import pandas as pd

df = pd.read_csv('path_to_dataset.csv')


In [ ]:
# Checking general info
df.info()

# Checking descriptive statistics
df.describe()


data cleaning

In [ ]:
missing_data = df.isnull().sum()

# Percentage of missing data
missing_percentage = (df.isnull().sum().sum() / df.size) * 100

# If less than 10% of the total data points are missing:
if missing_percentage < 10:
    df.dropna(inplace=True)
else:
    # Handle missing data based on the nature of the data
    for column in df.columns:
        # Assuming numeric columns can be filled with mean/median and categorical with mode
        if df[column].dtype == 'object':
            df[column].fillna(df[column].mode()[0], inplace=True)
        else:
            df[column].fillna(df[column].mean(), inplace=True)


Data Exploration: 
Exploring continuous variables:

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

continuous_columns = df.select_dtypes(include=['float64', 'int64']).columns

for column in continuous_columns:
    sns.boxplot(x='output_variable', y=column, data=df)
    plt.show()


Exploring categorical variables:

In [ ]:
categorical_columns = df.select_dtypes(include=['object']).columns

for column in categorical_columns:
    proportions = df.groupby(column)['output_variable'].value_counts(normalize=True)
    print(proportions)


In [ ]:
# Binning example
df['binned_column'] = pd.cut(df['continuous_column'], bins=[0, 10, 20, 30], labels=['0-10', '10-20', '20-30'])

# Dummy variables
df = pd.get_dummies(df, columns=['categorical_column'])


In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop('output_variable', axis=1)
y = df['output_variable']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

models = {
    'Logistic Regression': LogisticRegression(),
    'Naive Bayes': GaussianNB(),
    'KNN': KNeighborsClassifier(),
    'SVM': SVC(),
    'Decision Tree': DecisionTreeClassifier()
}

for name, model in models.items():
    f1_scores = cross_val_score(model, X_train, y_train, cv=10, scoring='f1')
    avg_f1 = f1_scores.mean()
    print(f"Average F1 Score for {name}: {avg_f1}")


Optimize based on Precision or Recall?
Your decision here will be based on the business objective:

Precision: It's the ratio of correctly predicted positive observations to the total predicted positives. High precision indicates that an algorithm returns more relevant results than irrelevant ones. Choose precision when the cost of false positives (wrongly predicted as positive) is high.

Example: In an email spam filter, you'd rather let some spam emails pass through (false negatives) than accidentally sending a legitimate email (false positive) to the spam folder.

Recall (Sensitivity): It indicates the ratio of correctly predicted positive observations to all the observations in the actual class. Choose recall when the cost of false negatives (wrongly predicted as negative) is high.

Example: In medical diagnoses, it's more acceptable to have false positives (and then run more tests) than to miss a person who has the disease (false negative).

In [ ]:
#Hyperparameter Tuning:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

# Assuming we're using SVM and tuning C, kernel, and gamma
param_dist = {
    'C': uniform(loc=0, scale=4),
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto', uniform(0.1, 1)]
}

svm = SVC()
r_search = RandomizedSearchCV(svm, param_distributions=param_dist, n_iter=100, scoring='f1', cv=5, n_jobs=-1)
r_search.fit(X_train, y_train)


In [ ]:
#Check Performance Metrics:
#After hyperparameter tuning, check the model's performance using different metrics:

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score

y_pred = r_search.best_estimator_.predict(X_test)

print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))
print("ROC AUC:", roc_auc_score(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))


Refine the Model:
If your model is still underperforming:

Feature Engineering: This involves coming up with new features based on domain knowledge, interactions between existing features, or even external data.

Feature Selection: Some features might be adding noise rather than value. You can use techniques like recursive feature elimination, feature importance from tree-based models, or correlation matrices to remove insignificant features.

Data Cleaning: There might be outliers or anomalies affecting the model's performance. Consider using robust scalers or manually removing these anomalies.

Gathering More Data: If possible, getting more data can help improve the model's generalization capabilities.

Model Stacking/Ensembling: This involves using multiple models together to get better performance than any single model. Techniques include bagging, boosting, and stacking.



The error message you received indicates that there's an issue with the data type in your dataset. The K-Nearest Neighbors (KNN) algorithm (and many other machine learning algorithms in Scikit-Learn) requires input features to be numeric. The error specifically points to a string value 'oppo', which suggests that at least one of your features is categorical with string values.

To address this issue, you'll need to preprocess your data to handle these categorical values. Here are some steps you can take:

Identify Categorical Features: Check which columns in your dataset are of type object (typically representing string values). You can do this using:

python
Copy code
categorical_features = X_train.select_dtypes(include=['object']).columns
print(categorical_features)
Encode Categorical Features: One of the common methods to handle categorical features is to use one-hot encoding, which will convert categorical variables into a format that works better with classification algorithms.

python
Copy code
from sklearn.preprocessing import OneHotEncoder

# One-hot encode categorical columns
encoder = OneHotEncoder(drop='first', sparse=False)  # `drop='first'` to avoid multicollinearity
encoded_features = encoder.fit_transform(X_train[categorical_features])
encoded_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(categorical_features))

# Drop the original categorical columns from X_train
X_train = X_train.drop(columns=categorical_features)

# Concatenate the one-hot encoded columns to X_train
X_train = pd.concat([X_train.reset_index(drop=True), encoded_df.reset_index(drop=True)], axis=1)
Repeat for Test Data: You need to apply the same transformation to your test data.

python
Copy code
encoded_features_test = encoder.transform(X_test[categorical_features])
encoded_df_test = pd.DataFrame(encoded_features_test, columns=encoder.get_feature_names_out(categorical_features))
X_test = X_test.drop(columns=categorical_features)
X_test = pd.concat([X_test.reset_index(drop=True), encoded_df_test.reset_index(drop=True)], axis=1)
Retrain the Model: Now that your training and testing data are properly encoded, you can re-run the fit method for your model.

python
Copy code
knn.fit(X_train, y_train)
Remember to always ensure that any transformation you apply to your training data is also applied to your test data, and any future data you intend to make predictions on.




